In [1]:
import pandas as pd
import altair as alt

# Получаем данные...

In [2]:
DATA_DIR = '/Users/kamiliakashafetdinova/mfk/'

In [3]:
df = pd.read_csv(f'{DATA_DIR}flights_NY.csv')

In [4]:
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance
0,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400
1,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416
2,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089
3,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576
4,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762


In [5]:
df.isna().any(axis=1).unique()

array([False,  True])

### Удаляю строки с пропущенными значениями

In [6]:
df = df.dropna()

In [7]:
df.isna().any(axis=1).unique()

array([False])

# Задание №1

In [8]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [9]:
df['is_positive_arr_delay'] = df['arr_delay'].apply(lambda x: 1 if x > 0 else 0)

In [10]:
pos_arr_delay_carrier = df.pivot_table(
    index='carrier', values=['is_positive_arr_delay', 'flight'], aggfunc={'is_positive_arr_delay': 'sum', 'flight': 'count'}, margins='All|'
).assign(pos_arr_delay_prob = lambda x: x['is_positive_arr_delay'] / x ['flight']).reset_index()

In [11]:
pos_arr_delay_all = sum(pos_arr_delay_carrier.query('carrier == "All"')['is_positive_arr_delay'])

In [12]:
pos_arr_delay_carrier[
    'pos_arr_delay_carr_weight'] = pos_arr_delay_carrier['is_positive_arr_delay'].apply(lambda x: x / pos_arr_delay_all)

In [13]:
pos_arr_delay_all_prob = sum(pos_arr_delay_carrier.query('carrier == "All"')['pos_arr_delay_prob'])

In [14]:
pos_arr_delay_carrier[
    'pos_arr_delay_carr_weight_prob'] = pos_arr_delay_carrier['pos_arr_delay_carr_weight'].apply(lambda x: x * pos_arr_delay_all_prob)

In [15]:
pos_arr_delay_carrier

,carrier,flight,is_positive_arr_delay,pos_arr_delay_prob,pos_arr_delay_carr_weight,pos_arr_delay_carr_weight_prob
0,9E,17294,6637,0.383775,0.049901,0.020275
1,AA,31947,10706,0.335118,0.080494,0.032705
2,AS,709,189,0.266573,0.001421,0.000577
3,B6,54049,23609,0.436807,0.177506,0.072122
4,DL,47658,16413,0.344391,0.123402,0.050140
5,EV,51108,24484,0.479064,0.184085,0.074795
6,F9,681,392,0.575624,0.002947,0.001198
7,FL,3175,1895,0.596850,0.014248,0.005789
8,HA,342,97,0.283626,0.000729,0.000296
9,MQ,25037,11693,0.467029,0.087915,0.035721


In [16]:
carr_pos_delay_chart = alt.Chart(pos_arr_delay_carrier.query('carrier != "All"')).mark_bar(
    size=30
).encode(
    x=alt.X('carrier').title('Авиакомпания'),
    y=alt.Y('pos_arr_delay_prob').title('Вероятность задержки')
).properties(
    title='Вероятности положительной задержки у авиакомпаний',
    height=200,
    width=900
)

carr_pos_delay_chart_marks = carr_pos_delay_chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    text=alt.Text('pos_arr_delay_prob', format=',.2f')
)

In [17]:
carr_pos_delay_chart + carr_pos_delay_chart_marks

alt.LayerChart(...)

In [18]:
carr_pos_delay_dist_chart = alt.Chart(pos_arr_delay_carrier.query('carrier != "All"')).mark_bar(
    size=30
).encode(
    x=alt.X('carrier').title('Авиакомпания'),
    y=alt.Y('pos_arr_delay_carr_weight_prob').title('Доля общей положительной задержки')
).properties(
    title='Распределение вероятности положительной задержки по авиакомпаниям',
    height=200,
    width=900
)

carr_pos_delay_dist_chart_marks = carr_pos_delay_dist_chart.mark_text(
    align='center',
    baseline='bottom',
    dy=-5
).encode(
    text=alt.Text('pos_arr_delay_carr_weight_prob', format=',.3f')
)

In [19]:
carr_pos_delay_dist_chart + carr_pos_delay_dist_chart_marks

alt.LayerChart(...)